## Ejercicio de Redes Neuronales 02

Usando las imagenes del directorio **`Dogs`**, construye una red neuronal que prediga la raza del perro.

- Tienes que leer y clasificar cada imagen, el directorio **`Dogs`** esta conformado por directorios con los nombres de las razas.

- Cada imagen tiene un tamaño diferente, para poder construir una red neuronal todas las imagenes deben de tener el mismo tamaño.

- El tamaño de las imagenes determinará el número de neuronas de la capa de entrada, y el número de razas determinará el número de neuronas de la capa de salida.

- Recuerda usar la función la función **`to_categorical()`** y "aplanar" las imagenes.

- Muestra la matriz de confusión del modelo.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import keras

# Dataset
from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import os

from sklearn.metrics import confusion_matrix
import cv2

In [2]:
os.getcwd()

'C:\\Users\\MAXHUB\\Downloads\\mod10-deep-learning'

In [3]:
os.listdir()

['.ipynb_checkpoints',
 '00.deep_learning_introduccion.ipynb',
 '01.redes_neuronales_perceptron.ipynb',
 '02.funciones_activacion.ipynb',
 '03.forward_propagation.ipynb',
 '04.back_propagation_hecho_en_clase.ipynb',
 '05.keras.ipynb',
 '06.neural_network_ejercicio_01.ipynb',
 '07.neural_network_ejercicio_02.ipynb',
 'Dogs',
 'kmnist_modelo.h5']

In [4]:
os.chdir("Dogs")

In [5]:
lista_x_train = list()
lista_x_test = list()
lista_y_train = list()
lista_y_test = list()

for directorio in os.listdir():

    if not(directorio.startswith(".")):
    
        if ("Test" in directorio):
            os.chdir(directorio)
            
            for imagen in os.listdir():
                imagen = cv2.imread(filename = imagen, flags = 0)
                lista_x_test.append(imagen)
                lista_y_test.append(directorio.split("_")[0])
            os.chdir("..")    
        
    
        else:
            os.chdir(directorio)
            
            for imagen in os.listdir():
                imagen = cv2.imread(filename = imagen, flags = 0)
                lista_x_train.append(imagen)
                lista_y_train.append(directorio)
            os.chdir("..") 
    
    
    
    #img[i] = cv2.imread(filename = "imagen_prueba_2.jpeg")

In [6]:
lista_tamaños = list()

for num, imagen in enumerate(lista_x_test + lista_x_train):
    
    lista_tamaños.append(imagen.shape)
    
lista_tamaños = np.array(lista_tamaños)

ancho = lista_tamaños[:, 0]
alto = lista_tamaños[:, 1]

In [7]:
ancho, alto = int(ancho.mean())//4, int(alto.mean())//4

In [8]:
for num, imagen in enumerate(lista_x_test):
    
    lista_x_test[num] = cv2.resize(src = imagen, dsize = (ancho, alto))

In [9]:
for num, imagen in enumerate(lista_x_train):
    
    lista_x_train[num] = cv2.resize(src = imagen, dsize = (ancho, alto))

In [10]:
X_train = np.array(lista_x_train)
X_test = np.array(lista_x_test)

In [11]:
# Para adaptar cada imagen a la red neuronal debemos transformalo en un vector, es decir, un elemento de 1 columna.

# flatten 
num_pixels = X_train.shape[1] * X_train.shape[2] 

# Train
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype("float32") # flatten training images

# Test
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype("float32") # flatten test images

In [12]:
X_train = X_train/255
X_test = X_test/255

In [13]:
dogs_dict = {raza : num for num, raza in enumerate(set(lista_y_train))}
dogs_dict

{'Doberman': 0, 'Labrador': 1, 'Yorkshire': 2, 'ShihTzu': 3}

In [14]:
y_train = to_categorical([dogs_dict[dog] for dog in lista_y_train])
y_test = to_categorical([dogs_dict[dog] for dog in lista_y_test])

y_train[0]

array([1., 0., 0., 0.], dtype=float32)

In [15]:
num_clases = len(y_train[0])
num_clases

4

In [16]:
num_pixels

11200

In [17]:
model = Sequential()

# Capa de Entrada
model.add(Dense(num_pixels, activation = "relu", input_shape = (num_pixels, )))

# Capas Ocultas
model.add(Dense(num_pixels//2, activation = "relu"))
model.add(Dense(num_pixels//4, activation = "relu"))
model.add(Dense(num_pixels//8, activation = "relu"))
model.add(Dense(num_pixels//16, activation = "relu"))
model.add(Dense(num_pixels//32, activation = "relu"))

# Capa de Salida (Si es un problema de clasificación multiple siempre debe terminar con "softmax")
model.add(Dense(num_clases, activation = "softmax"))

# Compilamos el modelo
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 11200)             125451200 
                                                                 
 dense_1 (Dense)             (None, 5600)              62725600  
                                                                 
 dense_2 (Dense)             (None, 2800)              15682800  
                                                                 
 dense_3 (Dense)             (None, 1400)              3921400   
                                                                 
 dense_4 (Dense)             (None, 700)               980700    
                                                                 
 dense_5 (Dense)             (None, 350)               245350    
                                                                 
 dense_6 (Dense)             (None, 4)                 1

In [19]:
# Entrenamos el modelo
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 20, verbose = 1)

# Metricas
scores = model.evaluate(X_test, y_test, verbose = 1)

Epoch 1/20
13/13 [==============================] - 26s 2s/step - loss: 22.8490 - accuracy: 0.2650 - val_loss: 1.7522 - val_accuracy: 0.2750
Epoch 2/20
13/13 [==============================] - 21s 2s/step - loss: 1.5399 - accuracy: 0.2650 - val_loss: 1.4278 - val_accuracy: 0.2500
Epoch 3/20
13/13 [==============================] - 16s 1s/step - loss: 1.4108 - accuracy: 0.2150 - val_loss: 1.4133 - val_accuracy: 0.2500
Epoch 4/20
13/13 [==============================] - 16s 1s/step - loss: 1.3902 - accuracy: 0.2625 - val_loss: 1.3268 - val_accuracy: 0.4750
Epoch 5/20
13/13 [==============================] - 18s 1s/step - loss: 1.3363 - accuracy: 0.3750 - val_loss: 1.3136 - val_accuracy: 0.3583
Epoch 6/20
13/13 [==============================] - 17s 1s/step - loss: 1.3374 - accuracy: 0.3100 - val_loss: 1.2919 - val_accuracy: 0.3667
Epoch 7/20
13/13 [==============================] - 17s 1s/step - loss: 1.3085 - accuracy: 0.3650 - val_loss: 1.3757 - val_accuracy: 0.2583
Epoch 8/20
13/13 [=

In [20]:
scores

[1.393373966217041, 0.2916666567325592]

In [21]:
y_hat = model.predict(X_test)
y_hat

array([[2.88294941e-01, 2.33139664e-01, 2.71432549e-01, 2.07132846e-01],
       [7.58369327e-01, 8.70433971e-02, 1.24113984e-01, 3.04733999e-02],
       [6.38665259e-02, 2.74310797e-01, 3.33047092e-01, 3.28775644e-01],
       [2.59124637e-01, 2.29530752e-01, 2.96308160e-01, 2.15036452e-01],
       [5.71679771e-01, 1.56494796e-01, 1.91239461e-01, 8.05859342e-02],
       [9.14468523e-03, 2.71696150e-01, 3.81920248e-01, 3.37238938e-01],
       [6.69826716e-02, 2.63817996e-01, 3.34420741e-01, 3.34778547e-01],
       [3.35547686e-01, 2.19429418e-01, 2.70616025e-01, 1.74406886e-01],
       [1.39254376e-01, 2.69379407e-01, 3.00135970e-01, 2.91230291e-01],
       [3.08870614e-01, 2.10711226e-01, 3.07685524e-01, 1.72732592e-01],
       [4.54705596e-01, 1.88769475e-01, 2.37333149e-01, 1.19191758e-01],
       [6.81261301e-01, 1.17303602e-01, 1.52370274e-01, 4.90648709e-02],
       [8.74407142e-02, 2.70055503e-01, 3.24287802e-01, 3.18215996e-01],
       [1.85700029e-01, 2.59669900e-01, 3.06401640e

In [22]:
y_hat = [np.argmax(i) for i in y_hat]
y_hat

[0,
 0,
 2,
 2,
 0,
 2,
 3,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 3,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 1,
 0,
 2,
 0,
 2,
 3,
 2,
 1,
 2,
 2,
 0,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 1,
 2,
 0,
 2,
 2,
 2,
 2,
 1,
 0,
 2]

In [23]:
y_test = [np.argmax(i) for i in y_test]
y_test

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2]

In [24]:
confusion_matrix(y_test, y_hat)

array([[15,  1, 13,  1],
       [ 8,  1, 19,  2],
       [11,  3, 16,  0],
       [ 2,  3, 22,  3]], dtype=int64)

In [25]:
print(f"Accuracy: {scores[1]}\nError: {1 - scores[1]}")

Accuracy: 0.2916666567325592
Error: 0.7083333432674408
